In [1]:
!pip install -r requirements.txt

/bin/bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by /bin/bash)

         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by bash)
  Using cached mysoc_dataset-0.3.0-py3-none-any.whl (9.3 kB)
  Using cached spacy-3.5.0-cp39-cp39-manylinux_2_17

  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached blis-0.7.9-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.2 MB)
  Using cached confection-0.0.4-py3-none-any.whl (32 kB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached tokenizers-0.13.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [4]:
import faculty.datasets as datasets
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import spacy
import re
from utils import clean2, is_pua
from sentence_transformers import SentenceTransformer
import pickle
%matplotlib inline

In [19]:
df = pd.read_excel('../data/raw/plans.xlsx')
drop_cols = ['search_link','type','charset','unfound','credit','date_retrieved','time_period','scope','status',
            'well_presented', 'baseline_analysis', 'notes', 'plan_path','file_type','website_url',
            'authority_code', 'authority_type', 'wdtk_id',
           'mapit_area_code', 'country', 'gss_code', 'county', 'region',
           'population', 'unfound','credit','homepage_mention', 'dedicated_page',
            'plan_due','title','title_checked','twitter_url','twitter_name','url','council']
df = df.drop(drop_cols,axis=1)
corpus = df[df['text'].notna()]

## cleaning

In [20]:
corpus['text'] = corpus['text'].apply(clean2)

/tmp/ipykernel_121348/3369171658.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['text'] = corpus['text'].apply(clean2)


In [21]:
text = corpus['text'].sample(1,random_state=5).iloc[0]
text

'cabinet 18 march 2020 report of councillor joe orson leader of the council climate emergency recommendation to council approach to climate emergency action plan corporate priority corporate strategy 2020 to 2024 protect our climate and enhance our rural, natural environment. relevant ward member date of consultation with ward choose date member exempt information no key decision no subject to call in no not key decision 1 summary 1.1 melton borough council set up the climate emergency policy development group in july 2019 to develop an action plan for consideration by cabinet and council by april 2020. 1.2 an action plan structure and content has been agreed by the group and cabinets consideration of its approach is sought. 1.3 the proposal is to develop the activity on a progressive stage process as follows  scoping work  baseline analysis  decarbonisation pathways  target setting  action plans  monitoring and evaluation further detail on the content and intention of these stages is 

In [7]:
def clean3(text):
    text = re.sub("[^a-z. ]",'',text)
    return text
# corpus['text'] = corpus['text'].apply(clean3)

/tmp/ipykernel_121348/843998945.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['text'] = corpus['text'].apply(clean3)


In [ ]:
import spacy
import pytextrank

text = corpus['text'].sample(1,random_state=5).iloc[0]

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")
# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")

doc = nlp(text)
print([w.text for w in doc if w.pos_ in ["PROPN","ADJ", "NOUN"]])
# examine the top-ranked phrases in the document
for phrase in doc._.phrases:
    print(phrase.text, phrase.rank, phrase.count)

In [ ]:
##